In [1]:
# importing libraries

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

import torch 
from torch import nn 
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm
from timeit import default_timer as timer

In [2]:
# Define the batch size and model save paths

BATCH_SIZE = 128
PATH = 'Models/Final/01_Direct_10M/'

In [3]:
# Loading the dataset

data = pd.read_csv('ik_dataset_4.csv')
data.head()

,q1 (deg),q2 (deg),q3 (deg),q4 (deg),q5 (deg),q6 (deg),x,y,z
0,-2.357874,0.537781,-0.179165,1.444722,0.403092,-6.954611,-0.139262,-0.174507,-0.243091
1,-0.269199,-1.255875,-2.729479,1.578971,1.526988,0.858804,-0.083841,0.090493,0.948361
2,0.399324,0.737099,0.269215,-1.295048,1.105814,-1.595081,0.005534,-0.058343,-0.095492
3,-0.630865,-0.393581,-0.569564,-2.048782,1.621882,-2.922093,0.554500,-0.476409,0.299512
4,2.624499,0.887666,-1.987815,-2.590494,0.107791,-3.599966,-0.603389,0.347365,-0.123526


In [4]:
# Normalising the data 

scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data))
data.columns = ['joint_0','joint_1','joint_2','joint_3','joint_4','joint_5','x','y','z']
data.head()

,joint_0,joint_1,joint_2,joint_3,joint_4,joint_5,x,y,z
0,0.124733,0.654063,0.749052,0.706941,0.596231,0.001913,0.420337,0.400196,0.130362
1,0.457156,0.140218,0.227187,0.726171,0.864541,0.561507,0.452021,0.551658,0.870414
2,0.563554,0.711163,0.840803,0.314498,0.763994,0.385761,0.503116,0.466590,0.222041
3,0.399595,0.387247,0.669166,0.206534,0.887196,0.290720,0.816958,0.227642,0.467391
4,0.917702,0.754298,0.378952,0.128939,0.525733,0.242171,0.154997,0.698475,0.204627


In [5]:
# Train test split

data_train, data_test = train_test_split(data, train_size=0.8, random_state=42)
data_train.shape, data_test.shape

((8000000, 9), (2000000, 9))

In [6]:
# Device agnostic code

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
device

'cuda'

In [7]:
# Converting the data to tensors
# We convert each column into seperate tensors and vertically stack them at the end so that more space is not used for each of the model

# Train
x_train = torch.Tensor(data_train['x'].to_numpy()).to(device)
y_train = torch.Tensor(data_train['y'].to_numpy()).to(device)
z_train = torch.Tensor(data_train['z'].to_numpy()).to(device)
joint_0_train = torch.Tensor(data_train['joint_0'].to_numpy()).to(device)
joint_1_train = torch.Tensor(data_train['joint_1'].to_numpy()).to(device)
joint_2_train = torch.Tensor(data_train['joint_2'].to_numpy()).to(device)
joint_3_train = torch.Tensor(data_train['joint_3'].to_numpy()).to(device)
joint_4_train = torch.Tensor(data_train['joint_4'].to_numpy()).to(device)
joint_5_train = torch.Tensor(data_train['joint_5'].to_numpy()).to(device)

# Test
x_test = torch.Tensor(data_test['x'].to_numpy()).to(device)
y_test = torch.Tensor(data_test['y'].to_numpy()).to(device)
z_test = torch.Tensor(data_test['z'].to_numpy()).to(device)
joint_0_test = torch.Tensor(data_test['joint_0'].to_numpy()).to(device)
joint_1_test = torch.Tensor(data_test['joint_1'].to_numpy()).to(device)
joint_2_test = torch.Tensor(data_test['joint_2'].to_numpy()).to(device)
joint_3_test = torch.Tensor(data_test['joint_3'].to_numpy()).to(device)
joint_4_test = torch.Tensor(data_test['joint_4'].to_numpy()).to(device)
joint_5_test = torch.Tensor(data_test['joint_5'].to_numpy()).to(device)

x_train.shape, x_test.shape

(torch.Size([8000000]), torch.Size([2000000]))

In [8]:

# Defining the model 

class InverseKinematicsABBIRB140(nn.Module):
    
    """ This is a model structure specifically designed to calculate the inverse kinematics of the industrial robot ABB IRB 140, but can be used for any robot
        This has a layer structure of input -> 64 -> 128 -> 64 -> output with non-linear activation and normalisation layers
    Args:
        input_features (int): defines the number of features given as input to the model
        output_features (int): defines the number of features the model outputs
    """
    
    def __init__(self, input_features, output_features):
        super().__init__()
        self.layer_stack_1 = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout()
        )
        self.layer_stack_2 = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout()
        )
        self.layer_stack_3 = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout()
        )
        self.layer_stack_4 = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout(),
            nn.Linear(in_features=128, out_features=output_features)
        )
    
    def forward(self, x):
        x = self.layer_stack_1(x)
        x = self.layer_stack_2(x)
        x = self.layer_stack_3(x)
        x = self.layer_stack_4(x)
        return x   
    

In [9]:
# Define the training step
def train_step(model: torch.nn.Module,
               batch: tuple,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device = device):
    
    xtrain, ytrain = batch
    xtrain, ytrain = xtrain.to(device), ytrain.to(device)
    
    model.train()
    ypreds = model(xtrain)
    loss = loss_fn(ypreds, ytrain)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item()

In [10]:
# Function to print the training time

def print_train_time(start:float, end:float, device:torch.device=None):
    total_time = end-start
    print(f"Train time on device {device} : {total_time:.4f} seconds")
    return total_time

In [11]:
# Define the testing step

def test_step(model: torch.nn.Module,
              batch: tuple,
              loss_fn: torch.nn.Module,
              device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
    
    xtest, ytest = batch
    xtest, ytest = xtest.to(device), ytest.to(device)
    
    model.eval()
    with torch.inference_mode():
        ypreds = model(xtest)
        loss = loss_fn(ypreds, ytest)
    
    return loss.item()

In [12]:
# Setting up model 0

NAME_0 = 'model_0.pth'
MODEL_SAVE_PATH_0 = PATH + NAME_0

train_dataset = TensorDataset(torch.stack((x_train, y_train ,z_train), dim=1), torch.stack((joint_0_train, joint_1_train, joint_2_train, joint_3_train, joint_4_train, joint_5_train,), dim=1))
test_dataset = TensorDataset(torch.stack((x_test, y_test ,z_test), dim=1), torch.stack((joint_0_test, joint_1_test, joint_2_test, joint_3_test, joint_4_test, joint_5_test, ), dim=1))

train_dataloader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle = False)

torch.manual_seed(42)
model_0 = InverseKinematicsABBIRB140(input_features=3, output_features=6).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

In [13]:
# Training model 0

torch.manual_seed(42)
train_time_start_model_0 = timer()
epochs = 6

for epoch in range(epochs):
    with tqdm(enumerate(train_dataloader), total=len(train_dataloader)) as t:

        train_loss = 0
        # Training loop
        for i, batch in t:
            batch_loss = train_step(model=model_0, batch=batch, loss_fn=loss_fn, optimizer=optimizer)
            train_loss += batch_loss
            t.set_description(f'Epoch [{epoch+1}/{epochs}] (Training)')
            t.set_postfix({
                'Train Batch loss': batch_loss,
                'Train loss': train_loss/(i+1)
            })
                
    with tqdm(enumerate(test_dataloader), total=len(test_dataloader)) as t:
        test_loss = 0
        # Testing loop
        for i, batch in t:
            batch_loss = test_step(model=model_0, batch=batch, loss_fn=loss_fn)
            test_loss += batch_loss
            t.set_description(f'Epoch [{epoch+1}/{epochs}] (Testing) ')
            t.set_postfix({
                'Test batch loss': batch_loss,
                'Test loss': test_loss /(i+1)
            })
            
    torch.save(obj=model_0.state_dict(), f=MODEL_SAVE_PATH_0)        
      
    print('---------------------------------------------------------------------------------------------------------------------------------------------------------')

train_time_end_model_0 = timer()
train_time_model_0 = print_train_time(start=train_time_start_model_0, end=train_time_end_model_0, device=device)

Epoch [1/6] (Testing) : 100%|██████████| 15625/15625 [00:45<00:00, 345.82it/s, Test batch loss=0.0558, Test loss=0.0591]


---------------------------------------------------------------------------------------------------------------------------------------------------------


Epoch [2/6] (Testing) : 100%|██████████| 15625/15625 [00:45<00:00, 345.07it/s, Test batch loss=0.0558, Test loss=0.0592]


---------------------------------------------------------------------------------------------------------------------------------------------------------


Epoch [3/6] (Testing) : 100%|██████████| 15625/15625 [00:45<00:00, 345.31it/s, Test batch loss=0.057, Test loss=0.0592] 


...

---------------------------------------------------------------------------------------------------------------------------------------------------------


Epoch [5/6] (Testing) : 100%|██████████| 15625/15625 [00:45<00:00, 344.97it/s, Test batch loss=0.0555, Test loss=0.059]


---------------------------------------------------------------------------------------------------------------------------------------------------------


Epoch [6/6] (Testing) : 100%|██████████| 15625/15625 [00:45<00:00, 342.85it/s, Test batch loss=0.0559, Test loss=0.059] 

---------------------------------------------------------------------------------------------------------------------------------------------------------
Train time on device cuda : 2887.9366 seconds
